# How to set a prior given T=1 nonzero effect?

Assume one nonzero effect $T = 1$. We investigate SuSiE performance when we have a pve such that SuSiE has certain amount of power e.g. $pve = 0.05$.

## Results

* If we overspecify a prior, i.e. set a prior $> 0.05$, we would have higher power and lower fdr. But the compensation is that the size of confidence set becomes larger. Also, if we take the top hit in each confidence set, we have $9\%$ probability getting wrong. 

* If we underspecify a prior, i.e. set a prior $< 0.05$, we have lower power compared to overspecification, and higher fdr. The size of confidence set is around 1. And the top-hit-rate is higher. 

The next question we want to ask is: what if the number of nonzero effects is not 1? Should we overspecify or underspecify a prior? 



In [29]:
dscout.summary[dscout.summary$pve==0.05,]

,effect_num,pve,prior,power,fdr,cs_size,cs_num,top_hit_rate,avg_purity
19,1,0.05,0.01,0.54,0.0357,1.107143,0.56,0.9643,0.9839111
85,1,0.05,0.02,0.60,0.0323,2.387097,0.62,0.9355,0.9601676
151,1,0.05,0.03,0.60,0.0323,1.838710,0.62,0.9355,0.9661933
217,1,0.05,0.05,0.60,0.0323,1.483871,0.62,0.9355,0.9709570
283,1,0.05,0.10,0.66,0.0294,3.382353,0.68,0.9118,0.9456457
349,1,0.05,0.20,0.66,0.0294,2.941176,0.68,0.9118,0.9456457
415,1,0.05,0.40,0.66,0.0294,2.735294,0.68,0.9118,0.9508725
481,1,0.05,0.50,0.66,0.0294,2.705882,0.68,0.9118,0.9508725
547,1,0.05,0.70,0.68,0.0286,3.800000,0.70,0.9143,0.9386454
613,1,0.05,0.90,0.68,0.0286,3.742857,0.70,0.9143,0.9386454


## Code details

In [23]:
dscout_Q2 = readRDS('dscout_Q2.rds')
dscout_Q2 = dscout_Q2[!is.na(dscout_Q2$sim_gaussian.output.file),]
dscout_Q2 = dscout_Q2[!is.na(dscout_Q2$susie_prior.output.file),]

In [24]:
dscout_df = data.frame(dscout_Q2$sim_gaussian.effect_num, dscout_Q2$sim_gaussian.pve, dscout_Q2$susie_prior.prior,
                       dscout_Q2$score.hit, dscout_Q2$score.signal_num, dscout_Q2$score.cs_medianSize,
                       dscout_Q2$score.top_hit, dscout_Q2$sim_gaussian.mean_corX, dscout_Q2$susie_prior.avg_purity)
names(dscout_df) = c('effect_num', 'pve', 'prior','hit', 'cs_num', 'cs_size', 'top_hit', 'corX', 'avg_purity')

In [25]:
power.summary = aggregate(hit ~ effect_num + pve + prior, dscout_df, sum)
power.summary$power = power.summary$hit / (power.summary$effect_num*50)
fdr.summary = aggregate(cs_num ~ effect_num + pve + prior, dscout_df, sum)
fdr.summary$fdr = round(1 - power.summary$hit / fdr.summary$cs_num, 4)
meannonzero = function(x){mean(x[x!=0])}
setsize.summary = aggregate(cs_size ~ effect_num + pve + prior, dscout_df, meannonzero)
tophit.summary = aggregate(top_hit ~ effect_num + pve + prior, dscout_df, sum)
tophit.summary$tophit_rate = round(tophit.summary$top_hit / fdr.summary$cs_num , 4)
dscout_df$avg_purity[is.na(dscout_df$avg_purity)]=0
purity.summary = aggregate(avg_purity ~ effect_num + pve + prior, dscout_df, meannonzero)

In [26]:
dscout.summary = data.frame(power.summary$effect_num, power.summary$pve, power.summary$prior,
                            power.summary$power, fdr.summary$fdr, 
                            setsize.summary$cs_size, fdr.summary$cs_num/50, 
                            tophit.summary$tophit_rate, purity.summary$avg_purity)
names(dscout.summary) = c('effect_num', 'pve', 'prior','power', 'fdr', 'cs_size', 'cs_num','top_hit_rate', 'avg_purity')

In [27]:
is.nan.data.frame <- function(x)
do.call(cbind, lapply(x, is.nan))
dscout.summary[is.nan(dscout.summary)] = 0

In [28]:
dscout.summary = dscout.summary[dscout.summary$effect_num==1, ]